In [9]:
# %load "ChatterBot.py"
# IMPORTANT: Load matplotlib and disable TKinter to deploy on Heroku.
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Any modules that utilize matplotlib (seaborn, etc.) must be imported below.
import json
import os
import time
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import tweepy
import pprint as pp

# Load environment variables.
from dotenv import load_dotenv
load_dotenv('.env')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


def get_commit_messages():
    compiled_commits = []
    for page in range(1, 10):
        url = f"https://api.github.com/users/droxey/events?type=PushEvent&public=true&distinct=true&page={page}"
        event_data = requests.get(url, headers={"User-Agent": "droxey"}).json()
        messages = [
            msg for msg in [
                commit[0].get("message", None) for commit in [
                    payload.get("commits", None) for payload in
                    [event.get("payload", None) for event in event_data]
                    if payload
                ] if commit
            ] if msg and msg not in compiled_commits
        ]
        for msg in messages:
            vs = analyzer.polarity_scores(msg)
            compiled_commits.append({'message': msg, 'score': vs})
    return compiled_commits


def main():
    print("Hello World!")
    commit_msgs = None
    auth = tweepy.OAuthHandler(
        os.getenv("CONSUMER_KEY"), os.getenv("CONSUMER_SECRET"))
    auth.set_access_token(
        os.getenv("ACCESS_TOKEN"), os.getenv("ACCESS_TOKEN_SECRET"))
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

    if commit_msgs is None:
        print("Fetching data...")
        commit_msgs = get_commit_messages()
    pp.pprint(commit_msgs)


if __name__ == "__main__":
    while True:
        main()
        time.sleep(60 * 60)
    print("Received shutdown signal. Goodbye!")


Hello World!
Fetching data...


AttributeError: 'str' object has no attribute 'get'